## Build a Wflow model from scratch

This notebook demonstrates how to prepare **Wflow** model from scratch using the command line interace (CLI).

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### hydroMT CLI build interface

Lets first check if the Wflow model is recognized by hydromt

In [ ]:
# this should return "wflow, wflow_seciment"
!hydromt --models

Using the `hydromt build` API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [ ]:
!hydromt build --help

### setup wflow model base layers 

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build wflow "./wflow_test_base" "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}" -r 0.00833 -v

The example above means the following: run **hydromt** with:

* `build wflow` : i.e. build a wflow model
* `./wflow_test_base` : output model folder
* `"{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}"` : derive a subbasin with its outlet at the given x,y coordinates and within the given bounds [xmin, ymin, xmax, ymax] (WGS84) snapped to a river with minimum stream order (strord) of 4. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/cli.html#region-options)
* `-r 0.0083333` : build the model at a 30 arc sec (~1km) resolution
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

As we did not specify a model configuration, only the basemaps with default parameters have been setup. To build a complete model we need the use a configuraton ini-file.

### Model setup configuration

The ini-file contains the model setup configuration and determines which components are build and in which sequence and sets optional arguments for each component. This configuration is passed to hydromt using `-i <path_to_ini_file>`. We have prepared several example ini-files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_wflow/tree/main/examples) and from the [docs(build_configuration)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow/build_configuration.html). 

Each header as shown between `[...]` (e.g. `[setup_basemaps]`) corresponds with a model component which are explained in the [docs(model_components)](https://deltares.github.io/hydromt_wflow/latest/user_guide/wflow/components.html). 

We will load the default wflow build ini file for inspection:

In [ ]:
fn_ini = 'wflow_build.ini'
with open(fn_ini, 'r') as f:
    txt = f.read()
print(txt)

### Data for model setup

The `source_name` and `<...>_fn` arguments correspond to a data source from the `DataCatalog`. The build this catalog a yml-file needs to be provided to hydromt using `-d <path_to_yml_file>`. 

Some example data for the Piave basin can be downloaded to `~/.hydromt_data/` and will be used for this example. An overview of the available data is provided [here](https://deltares.github.io/hydromt/latest/user_guide/data.html#available-global-datasets)
This example data is a based on the data which available from the Deltares p-drive. If you have acces to this drive, a pre-configured catalog file can be loaded using the `--dd` flag. 

More background how to write a data catalog yml file can be found in the [hydromt core docs](https://deltares.github.io/hydromt/latest/user_guide/data.html) 

### Setup complete wflow model 

In [ ]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build wflow "./wflow_test_full" "{'subbasin': [12.2051, 45.8331], 'strord': 4, 'bounds': [11.70, 45.35, 12.95, 46.70]}" -r 0.00833 -i wflow_build.ini -vv

With this example we build a complete wflow model including forcing. Compared to the previous **hydromt build** we have added:
* `-i wflow_build.ini` : setup configuration file using

Next we check which files have been created. The model root should contain two netcdf (.nc) files, one for the staticmaps and one for the forcing data, a wflow configuration (.toml) file. These files are sufficient to run the wflow model. In addition several geometry (.geojson) files are saved in the *staticgeoms* folder.  Finally, the setup process is logged in the hydromt.log file. 

In [ ]:
import os
root = 'wflow_test_full'
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith('.xml'):
            continue
        print(f' - {name}')

### Visualize and/or inspect model schematization

* The **wflow plot** example notebook contains scripts to visualize your model
* The **wflow nc to raster files** example notebook contains scripts to write the nc model files to a netcdf which can be used to inspect (and modify) the model layers in e.g. QGIS.